In [ ]:
from sim_util import load_tiff_sequence
import json

import skimage
import numpy as np
from scipy.spatial import KDTree
from scipy.optimize import linear_sum_assignment

def get_descriptor(spot, spots, neighbors_for_descriptor = 3):
    kd = KDTree(spots)
    ds, idxs = kd.query(spot, k=neighbors_for_descriptor+1)
    d_vecs = spot - spots[idxs[1:]]
    desc = d_vecs.flatten()
    return desc

def match_kd(descs_a, descs_b):
    kd = KDTree(descs_a)
    ds, idxes = kd.query(descs_b)
    return idxes


def match_la(descs_a, descs_b):
    
    descs_a = np.array(descs_a)
    descs_b = np.array(descs_b)
        
    n_spots_max = np.max([descs_a.shape[0], descs_b.shape[0]])
    n_spots_a = descs_a.shape[0]
    n_spots_b = descs_b.shape[0]
    
    # pad for empty assignment
    if (n_spots_b > n_spots_a):
        descs_a = np.concatenate((descs_a, descs_b[descs_a.shape[0]:]))
    if (n_spots_b < n_spots_a):
        descs_b = np.concatenate((descs_b, descs_a[descs_b.shape[0]:]))
    
    print (descs_a)
    print (descs_b)
    a = np.tile(descs_a, [n_spots_max, 1] )
    b = np.repeat(descs_b, n_spots_max, 0)
    ds = np.sqrt(np.sum((a-b)**2, axis=1)).reshape((n_spots_max, n_spots_max))

    _, idxes_la = linear_sum_assignment(ds)
    idxes_la[idxes_la>=n_spots_a] = -1
    return idxes_la[:n_spots_b]

In [ ]:
desc_img = load_tiff_sequence('/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-bbeam-x0-yall-zall-i2-l500/', 'bbeam_sim_c2.*')

In [ ]:
desc_img.shape

from skimage.feature import peak_local_max
import numpy as np
from scipy import ndimage as ndi

norm = desc_img / np.max(desc_img)

peaks = peak_local_max(ndi.gaussian_filter(desc_img, 1), threshold_rel=0.005, min_distance=7)
peaks = peaks * 4


descs = [get_descriptor(p, peaks, 4) for p in peaks]
peaks.shape

In [ ]:
import json
from functools import reduce
from operator import add
from argparse import Namespace

n = Namespace()

def_path = '/Volumes/davidh-ssd/mv-sim/sim_tissue/sim2.json'
with open(def_path, 'r') as fd:
    params = json.load(fd)
    
n.__dict__.update(params)
gt = reduce(add, [v['points'] for v in n.fields.values()])
np.array(gt).shape


descs_gt = [get_descriptor(p, np.array(gt), 4) for p in gt]
descs_gt

In [ ]:
idxes = match_la(descs, descs_gt)

for i_gt, i in enumerate(idxes):
    print(peaks[i], gt[i_gt])

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
%matplotlib inline
plt.rcParams['figure.figsize'] = [10,10]


plt.imshow(np.max(ndi.gaussian_filter(desc_img,1), axis=0)**0.5)
#for g in gt:
for g in [peaks[i] for i in idxes]:
#for g in peaks:
    e = Ellipse((g[2]//4, g[1]//4), 10, 10, color='r', fill=None)
    plt.gca().add_artist(e)
plt.show()

In [ ]:
from sim_util import load_tiff_sequence, save_as_sequence
import numpy as np
from itertools import product

signal_imgs = [load_tiff_sequence(
    '/Volumes/davidh-ssd/mv-sim/sim3/sim3-bbeam-x{}-yall-zall-i2-l500/'.format(i),
    'bbeam_sim_c1.*') for i in range(4)]

desc_imgs = [load_tiff_sequence(
    '/Volumes/davidh-ssd/mv-sim/sim3/sim3-bbeam-x{}-yall-zall-i2-l500/'.format(i),
    'bbeam_sim_c2.*') for i in range(4)]

In [ ]:
import numpy as np
from numpy.random import poisson
from itertools import product
from sim_util import load_tiff_sequence, save_as_sequence

n_tiles_x = 4
n_tiles_y = 4
n_tiles_z = 1
two_sided_illum = True

q_signal = 0.98
q_descs = 1.0

snr_signal = 20
snr_descs = 30

in_fpath = '/Volumes/davidh-ssd/mv-sim/sim5/sim5-bbeam-x{x}-y{y}-z{z}-i{ill}-l500/'
out_fpath = '/Volumes/davidh-ssd/mv-sim/sim5/intensity_adjusted/sim5-bbeam-x{x}-y{y}-z{z}-i{ill}-l500/'
add_poisson = True

signal_imgs = []
desc_imgs = []
for (x,y,z,ill) in product(range(n_tiles_x), range(n_tiles_y), range(n_tiles_z), [1] if not two_sided_illum else [1,2]):

    signal_img = load_tiff_sequence(in_fpath.format(x=x, y=y, z=z, ill=ill), 'bbeam_sim_c1.*')
    desc_img = load_tiff_sequence(in_fpath.format(x=x, y=y, z=z, ill=ill), 'bbeam_sim_c2.*')
    signal_imgs.append(signal_img)
    desc_imgs.append(desc_img)
    
    print('loaded tile x={} y={}, z={}, illum={}'.format(x,y,z,ill))
    
qs_signal = np.array([np.quantile(signal_img, q_signal) for signal_img in signal_imgs])
qs_desc = np.array([np.quantile(desc_img, q_descs) for desc_img in desc_imgs])
mq_signal = np.mean(qs_signal)
mq_desc = np.mean(qs_desc)

print('mean {}-quantile of signal images: {}'.format(q_signal, mq_signal))
print('mean {}-quantile of descriptor images: {}'.format(q_descs, mq_desc))

for (i,(x,y,z,ill)) in enumerate(product(range(n_tiles_x), range(n_tiles_y), range(n_tiles_z), [1] if not two_sided_illum else [1,2])):
    
    signal_img = signal_imgs[i]
    desc_img = desc_imgs[i]
    
    signal_img /= mq_signal
    desc_img /= mq_desc
    signal_img *= snr_signal**2
    desc_img *= snr_descs**2
    
    if add_poisson:
        signal_img = poisson(signal_img)
        desc_img = poisson(desc_img)
        
    out_dir = out_fpath.format(x=x, y=y, z=z, ill=ill)
    
    save_as_sequence(signal_img, out_dir, 'bbeam_sim_c1_z{plane}.tif')
    save_as_sequence(desc_img, out_dir, 'bbeam_sim_c2_z{plane}.tif')
    
    print('saved intensity adjusted tile x={} y={}, z={}, illum={}'.format(x,y,z,ill))
    


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

ps = poisson(signal_img / np.quantile(signal_img, q_signal) * snr_signal**2)

plt.imshow(np.max(desc_img, axis=0))
ps

In [ ]:
for i in range(len(signal_imgs)):
    print(np.quantile(signal_imgs[i], [0.1, 0.5, 0.9]))
    
print('descs')

for i in range(len(desc_imgs)):
    print(np.quantile(desc_imgs[i], [0.1, 0.5, 0.9, 1.0]))

In [ ]:
combd = np.stack(signal_imgs)
combd_desc = np.stack(desc_imgs)
print(np.quantile(combd, [0.1, 0.5, 0.9]))
print(np.quantile(combd_desc, [0.99, 0.999, 0.9999, 0.99999]))
combd.shape

In [ ]:
plt.imshow(np.max(desc_imgs[0]/3.41457848e-05, axis=0))
np.max(desc_imgs[0]/3.41457848e-05)

peak_local_max(desc_imgs[0], threshold_rel=0.5)

In [ ]:
plt.rcParams['figure.figsize'] = [15,15]

from calmutils.localization import detect_dog

peaks = detect_dog(desc_imgs[0], 0.002, 2.0)
#peaks = peak_local_max(ndi.gaussian_laplace(desc_imgs[0], 3), threshold_rel=0.05)
plt.imshow(np.max(desc_imgs[0], axis=0)**0.35)

for g in peaks:
    e = Ellipse((g[2], g[1]), 10, 10, color='r', fill=None)
    plt.gca().add_artist(e)
plt.show()

len(peaks)

In [ ]:
peaks

n = Namespace()

def_path = '/Volumes/davidh-ssd/mv-sim/sim3/sim3.json'
with open(def_path, 'r') as fd:
    params = json.load(fd)
    
n.__dict__.update(params)
gt = reduce(add, [v['points'] for v in n.fields.values()])

gt_1 = list(n.fields.values())[7]['points']

get_desc = [get_descriptor(g, np.array(gt_1), 9) for g in gt_1]

desc = [get_descriptor(p*2, np.array(peaks)*2, 9) for p in peaks]

idxes = match_kd(desc, get_desc)

plt.imshow(np.max(desc_imgs[0], axis=0)**0.35)

for i_gt, i in enumerate(idxes):
    g=peaks[i]
    print(peaks[i]*2, gt_1[i_gt])
    e = Ellipse((g[2], g[1]), 10, 10, color='r', fill=None)
    plt.gca().add_artist(e)
plt.show()

p1=[peaks[i]*2 for i in idxes]

from skimage.transform import AffineTransform
from skimage.measure import ransac

# FAIL: only 2d in skimage -> ImgLib2
np.array(gt_1).shape, np.array(p1).shape
ransac((np.array(gt_1), np.array(p1)), AffineTransform, 4, 5)


In [ ]:
import json

with open('/Volumes/davidh-ssd/mv-sim/sim3/sim3.json', 'r') as fd:
    params = json.load(fd)
    
params['fields']